In [1]:
# Imports
import tarfile
import zlib
from typing import List
from tika import parser
from nltk import sent_tokenize, wordpunct_tokenize
from nltk.stem.snowball import EnglishStemmer
from gensim.models.word2vec import Word2Vec

In [2]:
# Iterate through tar files

In [3]:
stemmer = EnglishStemmer()

def extract_sentences(text: str) -> List[List[str]]:
    """
    """
    sentence_list = [
        [stemmer.stem(t).lower() for t in wordpunct_tokenize(sentence) if t.isalnum()]
        for sentence in sent_tokenize(text)
    ]
    return sentence_list

extract_sentences("This is an example sentence.  And yet another here.")

[['this', 'is', 'an', 'exampl', 'sentenc'], ['and', 'yet', 'anoth', 'here']]

In [4]:
# Setup file
for year in range(1994, 2003):
    # Get file name and open
    file_name = "data/filings_10k_{0}.tar.gz".format(year)
    print(file_name)
    tar_file = tarfile.open(file_name)

    # Sample data
    sample_sentence_list = []

    # Iterate through members
    for j, tar_member in enumerate(tar_file.getmembers()):
        if j % 1000 == 0:
            print((year, j, len(sample_sentence_list)))

        # Skip non-files
        if not tar_member.isfile():
            continue

        # Parse real files
        try:
            # Read tar data
            member_buffer = zlib.decompress(tar_file.extractfile(tar_member).read())

            # Send to tika
            filing_buffer = parser.from_buffer(member_buffer, 'http://localhost:9998/tika')
            if 'content' in filing_buffer:
                filing_buffer = filing_buffer['content']
            else:
                continue

            # Parse
            #print((tar_member, len(tar_data), len(filing_buffer), len(sample_sentence_list)))

            # Get sentence list
            sample_sentence_list.extend(extract_sentences(filing_buffer))

        except Exception as e:
            print(e)
    
    print("beginning training...")
    # Train w2v CBOW model
    w2v_model_cbow = Word2Vec(sample_sentence_list, vector_size=200, window=20, min_count=10, workers=2)
    w2v_model_cbow.save("w2v_model_cbow_{0}".format(year))
    print("cbow trained.")

    # Train w2v SG model
    w2v_model_sg = Word2Vec(sample_sentence_list, vector_size=200, window=20, min_count=10, workers=2, sg=1)
    w2v_model_sg.save("w2v_model_sg_{0}".format(year))
    print("sg trained.")

data/filings_10k_1994.tar.gz


FileNotFoundError: [Errno 2] No such file or directory: 'data/filings_10k_1994.tar.gz'